<a href="https://colab.research.google.com/github/ivangd97/englishToFrenchTranslatorLSTM/blob/master/LSTM_Translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import collections

import helper
import numpy as np


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6941505193293344568
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16775184024693274891
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1070979409812975863
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15956161332
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15132563382935361212
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [0]:
def load_data(file_name):
  file_list = drive.ListFile({'q': "'162MC5jofogq8VjFufnkWvFeR5TdTdO4S' in parents"}).GetList()

  for f in file_list:
    if f['title'] == file_name:
      file_id = f['id']
      downloaded = drive.CreateFile({'id': file_id})
      return downloaded.GetContentString().split('\n')

In [0]:
english_sentences = load_data('small_vocab_en')
french_sentences = load_data('small_vocab_fr')

In [19]:
from collections import Counter

english_words = [word for sentence in english_sentences for word in sentence.split()]
french_words = [word for sentence in french_sentences for word in sentence.split()]

print('How many English words? {}'.format(len(english_words)))
print('How many unique English words? {}'.format(len(Counter(english_words))))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*Counter(english_words).most_common(10)))[0]) + '"')
print()
print('How many French words? {}'.format(len(french_words)))
print('How many unique French words? {}'.format(len(Counter(french_words))))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*Counter(french_words).most_common(10)))[0]) + '"')

How many English words? 1823250
How many unique English words? 227
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

How many French words? 1961295
How many unique French words? 355
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [20]:
from keras.preprocessing.text import Tokenizer

def tokenize(sentences):
  """
  return: Tuple (tokenized sentences, tokenizer used)
  """
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentences)
  return tokenizer.texts_to_sequences(sentences), tokenizer

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [21]:
from keras.preprocessing.sequence import pad_sequences

def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {} in length of {}'.format(np.array(token_sent), len(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9] in length of 9
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17] in length of 10
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21] in length of 5
  Output: [18 19  3 20 21  0  0  0  0  0]


In [22]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [23]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [24]:
import warnings; warnings.simplefilter('ignore')
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    rnn = Dropout(0.15)(rnn)
    rnn = GRU(128, return_sequences=True)(rnn)
    rnn = Dropout(0.17)(rnn)
    rnn = GRU(128, return_sequences=True)(rnn)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size = 500, epochs = 100, validation_split = 0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/100
110288/110288 [==============================] - 21s 194us/step - loss: 2.5768 - acc: 0.4757 - val_loss: nan - val_acc: 0.5559
Epoch 2/100
110288/110288 [==============================] - 19s 173us/step - loss: 1.6704 - acc: 0.5830 - val_loss: nan - val_acc: 0.6080
Epoch 3/100
110288/110288 [==============================] - 19s 171us/step - loss: 1.3828 - acc: 0.6150 - val_loss: nan - val_acc: 0.6327
Epoch 4/100
110288/110288 [==============================] - 19s 174us/step - loss: 1.2318 - acc: 0.6410 - val_loss: nan - val_acc: 0.6600
Epoch 5/100
110288/110288 [==============================] - 19s 172us/step - loss: 1.1289 - acc: 0.6604 - val_loss: nan - val_acc: 0.6862
Epoch 6/100
110288/110288 [==============================] - 19s 170us/step - loss: 1.0580 - acc: 0.6744 - val_loss: nan - val_acc: 0.6987
Epoch 7/100
110288/110288 [==============================] - 19s 171us/step - loss: 1.0000 - acc: 0.6875 - val_loss

In [0]:
print(tmp_x[:1])

[[[17]
  [23]
  [ 1]
  [ 8]
  [67]
  [ 4]
  [39]
  [ 7]
  [ 3]
  [ 1]
  [55]
  [ 2]
  [44]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]
  [ 0]]]
